In [1]:
from copy import deepcopy
from dataclasses import dataclass
import multiprocessing as mp
from re import T
import torchvision.models as models
import torch
import copy
import torch.nn.functional as F
#from model import net
import random
import multiprocessing as mp
import os
import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
from torchvision import datasets,transforms
import random
from torch import optim,ceil
import torch.nn.functional as F
from torch import nn
import pickle
import numpy as np
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from pandas import DataFrame

In [2]:
from sklearn.metrics import accuracy_score

In [3]:

from torch.utils.data import Dataset,DataLoader
import numpy as np

class plantdisease(Dataset):
    def __init__(self, df,
                  img_dir='data/oneFolder'
                 , isTest=False):
        self.img_labels=df
        self.img_dir='../../Data/DataResized512/'
       # self.train, self.test = train_test_split(self.img_labels, test_size=0.05)

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
            img_path = os.path.join(self.img_dir, self.img_labels['image'][idx])

            image = (read_image(img_path).float())
            self.transform=transforms.Compose([transforms.Resize((342,512)),
        transforms.Normalize((124.4455, 159.9584, 104.1832), (47.1528, 41.4626, 49.2424))])
            image = self.transform(image)
            label = self.img_labels['labels'][idx]
            label=np.fromstring(label[1:-1], dtype=np.int,sep=',')
            return image, label

In [4]:
df=pd.read_csv('../../Data/datafull.csv')
df: DataFrame=df.drop('Unnamed: 0',axis=1)
train,valid_test=train_test_split(df,test_size=0.2)
valid,test=train_test_split(valid_test,test_size=0.5)

train_data=plantdisease(df=train.reset_index())
test_data=plantdisease(df=test.reset_index())
valid_data=plantdisease(df=valid.reset_index())

model=models.mobilenet_v2(pretrained=True)
model.classifier[1]=(torch.nn.Linear(in_features=1280, out_features=6, bias=True))

In [5]:

from torch.utils.data import Dataset,DataLoader
import numpy as np

class plantdisease(Dataset):
    def __init__(self, df,
                  img_dir='data/oneFolder'
                 , isTest=False):
        self.img_labels=df
        self.img_dir='../../Data/DataResized512/'
       # self.train, self.test = train_test_split(self.img_labels, test_size=0.05)

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
            img_path = os.path.join(self.img_dir, self.img_labels['image'][idx])

            image = (read_image(img_path).float())
            self.transform=transforms.Compose([transforms.Resize((342,512)),
        transforms.Normalize((124.4455, 159.9584, 104.1832), (47.1528, 41.4626, 49.2424))])
            image = self.transform(image)
            label = self.img_labels['labels'][idx]
            label=np.fromstring(label[1:-1], dtype=np.int,sep=',')
            return image, label

In [6]:
#Helper
def cfs_matrix_calculate(y,pred):
  pred=torch.sigmoid(pred)
  pred=torch.round(pred)
  
  
  matrix=0
  for i in range(0,y.shape[0]):
    matrix+=confusion_matrix(y[i], pred[i],labels=[1, 0])
  return matrix

In [7]:
from tqdm import tqdm
class Client():
    def __init__(self,id,dataset,model,config) -> object:
        self.id=id
        self.dataset=dataset
        self.model=model
        self.criterion=torch.nn.BCEWithLogitsLoss()
        
        self.config=config
    def __len__(self) -> int:
        return len(self.dataset)
    def __str__(self) -> str:
        print('Client ID: {}', self.id)
        
    def get_config(self):
        print(self.config)
        return self.config
    
    def update_model_params(self,params):
        self.model.load_state_dict(params)

    def metric(self,y,pred):
        pred=torch.sigmoid(pred)
        pred=torch.round(pred)
        return accuracy_score(y.float(), pred.detach())

    def wandb_log(self,log={}):
        try:
            wandb.log(log)
        except:
            print('Wandb logging error {}', self.id)        
    
    def train_gpu(self,round):
        optimizer=optim.Adam(self.model.parameters(),lr=self.config['learning_rate'],weight_decay=self.config['weight_decay'])
        dataloader=torch.utils.data.DataLoader(self.dataset,
                                         batch_size=int(self.config['batch_size']),
                                         shuffle=True)

        train_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        self.model.to(train_device)

        print('Client {} start Training'.format(self.id))

        # run=wandb.init(group="federated_round: "+str(round), job_type="Client ID: "+str(self.id), project=PROJNAME)
        
        for epoc in (range(self.config['epoch'])):
            epoch_loss=0.0
            step=0
            epoch_accuracy=[]
            for x,y in iter(dataloader):
                x,y=x.to(train_device),y.to(train_device)
                pred=self.model(x)

                loss=self.criterion(pred,y.float())

                x,y=x.cpu(),y.cpu()
                pred=pred.cpu().detach()
                epoch_loss+=loss.item()
                step+=1
                epoch_accuracy.append(self.metric(y,pred))
                

                optimizer.zero_grad(set_to_none=True)
                loss.backward()
                optimizer.step()
            accuracy=np.mean(epoch_accuracy)
            # print('Epoch : {} , loss: {}, accuracy: {}'.format(epoc,(epoch_loss/step),accuracy))

            # self.wandb_log({
            #                 'epoc':epoc,
            #                 'loss':(epoch_loss/step),
            #                 'accuracy':accuracy
            #                 })
        self.model=self.model.cpu()
        
        # run.finish()
        

In [8]:

class Server():
    def __init__(self,model=models.mobilenet_v2(pretrained=True),dataset=None,eval_dataset=None,config={}):
        self.client_list=[]
        self.model= model
        self.model.classifier[1]=(torch.nn.Linear(in_features=1280, out_features=6, bias=True))
        
        self.dataset=dataset
        self.eval_dataset=eval_dataset
        self.round_dices=[]

        self.best_accuracy=0.0
        
        self.config=config
    def __len__(self) -> int:
        return len(self.client_list)

    def get_config(self):
        print(self.config)
        return self.config

    def split_dataset_1label(self):
        list_dataset=np.array_split(self.dataset,self.config['server']['num_client'])
        list_dataset=map(lambda df:  plantdisease(df.reset_index(drop=True)),list_dataset)
        return list_dataset
    
    def generate_clients(self):
        dataset=self.split_dataset_1label()
        id=0
        for shard in dataset:
            self.client_list.append(Client(id=id,model=copy.deepcopy(self.model),dataset=shard,config=self.config['client']))
            id+=1

    def collect_clients_params(self):
        client_params=[]
        for index in range(0,len(self.round_dices)):
            client_params.append(self.client_list[index].model.state_dict())
        return client_params

    def update_client_model(self,params):
        for client in self.client_list:
            client.model.load_state_dict(copy.deepcopy(params))
        self.model.load_state_dict(copy.deepcopy(params))
        print('Clients updated ! Ready for next round')    

    def fedAvg(self):
        total_len=0
        for index in self.round_dices:
            total_len+=len(self.client_list[index])

        global_model_weights = copy.deepcopy(self.model.state_dict())
        for key in global_model_weights:
            global_model_weights[key] = torch.zeros_like(
                global_model_weights[key])
        
        #server_params=copy.deepcopy(self.centralModel.parameters())

        coef=torch.tensor([len(self.client_list[index])/total_len for index in self.round_dices])
        for index in range(0,len(self.round_dices)):
            local_model_weights = copy.deepcopy(self.client_list[index].model.state_dict())
            for key in global_model_weights:
                if str(global_model_weights[key].type())!='torch.LongTensor':

                    global_model_weights[key] += (coef[index] * local_model_weights[key])
        return global_model_weights

    def metric(self,y,pred):
        pred=torch.sigmoid(pred)
        pred=torch.round(pred)
        return accuracy_score(y.float(), pred.detach())

    def wandb_log(self,log={}):
        try:
            wandb.log(log)
        except:
            print('Wandb logging error server', )
            
    def evaluate(self,dataset,round):
        self.model.eval()
        criterion= torch.nn.BCEWithLogitsLoss()

        dataloader=torch.utils.data.DataLoader(dataset,batch_size=int(10),shuffle=True)
        run=wandb.init(group="federated_Evaluation",job_type='Evaluation', project=PROJNAME)

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        test_loss=0.0
        step=0
        test_accuracy=[]
        self.model=self.model.to(device)

        with torch.no_grad():
            for x,y in dataloader:
                x,y=x.to(device),y.to(device)
                pred=self.model(x)
                loss=criterion(pred,y.float())

                x,y=x.cpu(),y.cpu()
                pred=pred.cpu().detach()

                test_loss+=loss.item()
                step+=1        
                pred=torch.sigmoid(pred)
                pred=torch.round(pred)
                test_accuracy.append(self.metric(y,pred))
                
        accuracy=np.mean(test_accuracy)
        loss=test_loss/step
        if(accuracy>self.best_accuracy):
            model_log = wandb.Artifact('best_eval_accuracy', type='model')
            try:
                self.best_accuracy=accuracy
                model_log.add_file(self.save_model())
                run.log_artifact(model_log)
                print('model saved, best accuracy: {}'.format(self.best_accuracy))
            except:
                print('save model failed')
        print('Evaluation successfully, ')
        print('Round : {} ,Accuracy : {} loss : {} '.format(round,accuracy,loss))
        self.wandb_log({
                'round':round,
                'loss':loss,
                'accuracy':accuracy
                })
        self.model=self.model.to('cpu')
        return accuracy
    
    def save_model(self):
        file_name='bestmodel.h5'
        torch.save(self.model.state_dict(), file_name)
        return file_name
    
    def round_train(self):
        from random import sample
        for round in range(0,self.config['server']['num_round']):
            
            self.round_dices=sample(range(0,len(self)),self.config['server']['num_client_round'])
            print('Round {}',format(round))
            print('Including clients: {}'.format(self.round_dices))
            for index in self.round_dices:
                self.client_list[index].train_gpu(round=round)
            print('Finish training. Collecting parameters...')
            self.collect_clients_params()
            params=self.fedAvg()
            self.update_client_model(params=params)
            acc=self.evaluate(self.eval_dataset,round=round)

                
            
            
                

In [9]:

CONFIG={
    'client':{
    'learning_rate':0.00006824,
    'batch_size':16,
    'weight_decay':0.000001,
    'epoch':8,
    },
    'server':{
        'num_client':100,
        'num_client_round':25,
        'num_round':50,
    }
}



In [10]:
os.environ["WANDB_API_KEY"] = '761f77141558010f4d706f726fcf122aac0aae23'
os.environ["WANDB_MODE"]="offline"

PROJNAME='fed_train27_25_1label'
import wandb
df=pd.read_csv('../../Data/datafull.csv')
df: DataFrame=df.drop('Unnamed: 0',axis=1)

def defineLable(row):
    count=0
    for cols in row.index[1:-1]:
        if row[cols]!=0:
            return count
        else:
            count+=1
df['one_label']=df.apply(defineLable,axis=1)



train,valid_test=train_test_split(df,test_size=0.2)
valid,test=train_test_split(valid_test,test_size=0.5)

train.sort_values('one_label',inplace=True)
valid.sort_values('one_label',inplace=True)
test.sort_values('one_label',inplace=True)

train_data=plantdisease(df=train.reset_index())
test_data=plantdisease(df=test.reset_index())
valid_data=plantdisease(df=valid.reset_index())

model=models.mobilenet_v2(pretrained=True)
model.classifier[1]=(torch.nn.Linear(in_features=1280, out_features=6, bias=True))


# note that we define values from `wandb.config` instead 
# of defining hard values

config = {
    "lr": 0.00006824,
    "batch_size":16,
    "epoch":20
}
import warnings 
warnings.filterwarnings('ignore')
train=train.reset_index(drop=True)
ser=Server(dataset=train,eval_dataset=test_data,config=CONFIG)
ser.generate_clients()
ser.round_train()



Round {} 0
Including clients: [31, 15, 36, 59, 28, 4, 96, 74, 17, 71, 46, 73, 70, 62, 5, 63, 57, 13, 61, 33, 14, 85, 98, 3, 90]
Client 31 start Training
Client 15 start Training
Client 36 start Training
Client 59 start Training
Client 28 start Training
Client 4 start Training
Client 96 start Training
Client 74 start Training
Client 17 start Training
Client 71 start Training
Client 46 start Training
Client 73 start Training
Client 70 start Training
Client 62 start Training
Client 5 start Training
Client 63 start Training
Client 57 start Training
Client 13 start Training
Client 61 start Training
Client 33 start Training
Client 14 start Training
Client 85 start Training
Client 98 start Training
Client 3 start Training
Client 90 start Training
Finish training. Collecting parameters...
Clients updated ! Ready for next round


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


model saved, best accuracy: 0.09946524064171124
Evaluation successfully, 
Round : 0 ,Accuracy : 0.09946524064171124 loss : 0.6474301996715566 
Round {} 1
Including clients: [9, 69, 20, 15, 93, 30, 89, 91, 18, 84, 66, 70, 85, 29, 55, 33, 72, 64, 0, 96, 48, 17, 45, 80, 21]
Client 9 start Training
Client 69 start Training
Client 20 start Training
Client 15 start Training
Client 93 start Training
Client 30 start Training
Client 89 start Training
Client 91 start Training
Client 18 start Training
Client 84 start Training
Client 66 start Training
Client 70 start Training
Client 85 start Training
Client 29 start Training
Client 55 start Training
Client 33 start Training
Client 72 start Training
Client 64 start Training
Client 0 start Training
Client 96 start Training
Client 48 start Training
Client 17 start Training
Client 45 start Training
Client 80 start Training
Client 21 start Training
Finish training. Collecting parameters...
Clients updated ! Ready for next round


accuracy,▁
loss,▁
round,▁
accuracy,0.09947
loss,0.64743
round,0


model saved, best accuracy: 0.1606951871657754
Evaluation successfully, 
Round : 1 ,Accuracy : 0.1606951871657754 loss : 0.5950411083226536 
Round {} 2
Including clients: [67, 37, 24, 74, 25, 48, 14, 63, 12, 6, 49, 36, 18, 44, 20, 69, 97, 71, 85, 61, 73, 54, 11, 87, 93]
Client 67 start Training
Client 37 start Training
Client 24 start Training
Client 74 start Training
Client 25 start Training
Client 48 start Training
Client 14 start Training
Client 63 start Training
Client 12 start Training
Client 6 start Training
Client 49 start Training
Client 36 start Training
Client 18 start Training
Client 44 start Training
Client 20 start Training
Client 69 start Training
Client 97 start Training
Client 71 start Training
Client 85 start Training
Client 61 start Training
Client 73 start Training
Client 54 start Training
Client 11 start Training
Client 87 start Training
Client 93 start Training
Finish training. Collecting parameters...
Clients updated ! Ready for next round


...

Evaluation successfully, 
Round : 47 ,Accuracy : 0.17700534759358288 loss : 1.2370807421717414 
Round {} 48
Including clients: [20, 72, 84, 70, 21, 93, 19, 51, 82, 92, 6, 10, 80, 31, 91, 0, 43, 48, 34, 25, 18, 64, 98, 17, 94]
Client 20 start Training
Client 72 start Training
Client 84 start Training
Client 70 start Training
Client 21 start Training
Client 93 start Training
Client 19 start Training
Client 51 start Training
Client 82 start Training
Client 92 start Training
Client 6 start Training
Client 10 start Training
Client 80 start Training
Client 31 start Training
Client 91 start Training
Client 0 start Training
Client 43 start Training
Client 48 start Training
Client 34 start Training
Client 25 start Training
Client 18 start Training
Client 64 start Training
Client 98 start Training
Client 17 start Training
Client 94 start Training
Finish training. Collecting parameters...
Clients updated ! Ready for next round


accuracy,▁
loss,▁
round,▁
accuracy,0.17701
loss,1.23708
round,47


Evaluation successfully, 
Round : 48 ,Accuracy : 0.17834224598930487 loss : 1.2371604955770115 
Round {} 49
Including clients: [90, 18, 52, 59, 89, 14, 70, 39, 6, 73, 32, 10, 24, 62, 44, 92, 50, 95, 84, 86, 42, 41, 43, 47, 29]
Client 90 start Training
Client 18 start Training
Client 52 start Training
Client 59 start Training
Client 89 start Training
Client 14 start Training
Client 70 start Training
Client 39 start Training
Client 6 start Training
Client 73 start Training
Client 32 start Training
Client 10 start Training
Client 24 start Training
Client 62 start Training
Client 44 start Training
Client 92 start Training
Client 50 start Training
Client 95 start Training
Client 84 start Training
Client 86 start Training
Client 42 start Training
Client 41 start Training
Client 43 start Training
Client 47 start Training
Client 29 start Training
Finish training. Collecting parameters...
Clients updated ! Ready for next round


accuracy,▁
loss,▁
round,▁
accuracy,0.17834
loss,1.23716
round,48


Evaluation successfully, 
Round : 49 ,Accuracy : 0.1788770053475936 loss : 1.2238283541432038 


In [11]:
df[df['complex']>0]

,image,complex,frog_eye_leaf_spot,healthy,powdery_mildew,rust,scab,labels,one_label
1,8002cb321f8bfcdf.jpg,1,1,0,0,0,1,"[1, 1, 0, 0, 0, 1]",0
4,800cbf0ff87721f8.jpg,1,0,0,0,0,0,"[1, 0, 0, 0, 0, 0]",0
8,801f78399a44e7af.jpg,1,0,0,0,0,0,"[1, 0, 0, 0, 0, 0]",0
10,802291cee9fec9f4.jpg,1,0,0,0,0,0,"[1, 0, 0, 0, 0, 0]",0
28,803e3bd17a16e65c.jpg,1,0,0,0,0,0,"[1, 0, 0, 0, 0, 0]",0
...,...,...,...,...,...,...,...,...,...
18585,ffd4c5088f4605a7.jpg,1,0,0,0,0,0,"[1, 0, 0, 0, 0, 0]",0
18595,ffdc70c492370f02.jpg,1,0,0,0,0,0,"[1, 0, 0, 0, 0, 0]",0
18597,ffdef27b820081c4.jpg,1,0,0,0,0,0,"[1, 0, 0, 0, 0, 0]",0
18604,ffe1e03542661f22.jpg,1,0,0,0,0,0,"[1, 0, 0, 0, 0, 0]",0


In [12]:
print('train')

train


In [13]:
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()


free_gpu_cache()

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 33% | 91% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  8% |  7% |
